<div id='loot_liquidity_outline'> </div>


* # outline
    * imports
    * setup web3 module
    * ```LiquidityLooter``` class definition
        * getting price
        * compare to current pool,
            * if more than  

[back to top](#loot_liquidity_outline) 

# setup 

In [1]:
%load_ext autoreload

In [55]:
%autoreload

# setup for local imports 
import sys
sys.path.insert(0, '..')
sys.path.insert(0, '../..')

# local packages
from config import params
from utils import ( get_secret, 
                    get_wallet_key,
                    get_public_address,
                    get_contract_abi,
                    convert_asset_wei_to_human_amount,
                    convert_asset_human_amount_to_wei,
                    get_actual_bags_gns_weth,
                    initialize_bags_dict,
                    get_tick_spacing_for_fee,
                    get_current_price_and_tick,
                    calculate_tick_from_price,
                    calculate_price_from_tick,
                    get_tick_range,
                    )

# pypi package imports
from copy import deepcopy 
import datetime
from dotenv import load_dotenv
from eth_account import Account
import json
import math
import requests
from web3 import Web3, EthereumTesterProvider

load_dotenv() 



True

In [3]:
params['port'] = params['operating_wallets']['GNS_ETH_2_90']
params['arbiscan_api_key'] =  get_secret("ARBISCAN_API")

# ### standalone objects 
# 
# # 
# # ####
private_key = get_wallet_key(wallet=params['port']['wallet'])  # ### this stays out of state_dict, as that may get pickled. only thing that stays out of everyting 
w3 = Web3(Web3.HTTPProvider(get_secret("RPC_URL")))
# # ### 
# # 
# 
params['port']['address'] = get_public_address(private_key)

state_dict = {} 

state_dict['bags'] = initialize_bags_dict(public_address=params['port']['address'], base='GNS', quote='WETH', w3=w3)
state_dict['pool'] = {'address': params['pool_addresses'][params['port']['pair']]} 
state_dict['pool']['checksum_address'] = Web3.to_checksum_address(state_dict['pool']['address'])

try:
    state_dict['pool']['abi'] = get_contract_abi(params['arbiscan_api_key'], state_dict['pool']['checksum_address'])
    print("ABI fetched successfully.")
except Exception as e:
    print(str(e))

ABI fetched successfully.


#### getting the public address 

### token balances

### getting token balances (note that not making one function for both because there is an ABI issue for WETH
* I would rather manually handle it till I am familiar with the nature of these objects and am confident that the unified function I built will do the job

In [4]:
state_dict['bags']['actual']

{'wei_format': {'base': 3000000000000000267, 'quote': 99108453379737797},
 'human_format': {'base': 3.0000000000000004, 'quote': 0.0991084533797378}}

In [5]:
state_dict['bags']['eth']

{'wei_format': 23000000000000000, 'human_format': Decimal('0.023')}

# working with uniswap 

In [6]:
# # Convert the pool address to checksum format
# pool_checksum_address = Web3.to_checksum_address(pool_address)

# # Create a contract object using the ABI and the pool's address
# # pool_contract = self.web3.eth.contract(address=checksum_address, abi=abi)
# pool_contract = w3.eth.contract(address=pool_checksum_address, abi=pool_abi)

# # Fetching slot0 which contains the current price and other data
# slot0 = pool_contract.functions.slot0().call()
# current_price = slot0[0]

"""slot0 = [3682675051650787857010074191, -61377, 3046, 7200, 7200, 0, True]"""

'slot0 = [3682675051650787857010074191, -61377, 3046, 7200, 7200, 0, True]'

In [11]:
state_dict['pool']['tick_spacking'] = get_tick_spacing_for_fee(params['port']['pair_fee'])

state_dict['pool']['w3'] = w3.eth.contract(address=state_dict['pool']['checksum_address'], abi=state_dict['pool']['abi'])

state_dict['pool']['sqrt_price_x96'], state_dict['pool']['current_tick'] = get_current_price_and_tick(state_dict['pool']['w3'])

state_dict['pool']['desired_tick_range'] = get_tick_range(state_dict['pool']['current_tick'], params['port']['lp_width'], state_dict['pool']['tick_spacking'])

print("Relevant tick range:", list(state_dict['pool']['desired_tick_range']))

Relevant tick range: [-61680, -61620, -61560, -61500, -61440, -61380, -61320]


In [23]:
price_range_low = calculate_price_from_tick(state_dict['pool']['desired_tick_range'][0])
price_range_high = calculate_price_from_tick(state_dict['pool']['desired_tick_range'][-1])
price_range_low

0.002096068942693643

In [24]:
price_range_high = calculate_price_from_tick(state_dict['pool']['desired_tick_range'][-1])
price_range_high

0.0021728982131318303

In [25]:
calculate_price_from_tick(state_dict['pool']['current_tick'])

0.0021422626177395425

#### figuring out what proportion of liquidity goes into each asset 

In [31]:
def calculate_liquidity_amounts(state_dict):
    """
    Calculate the amount of base and quote assets to be provided for liquidity.

    :param lower_tick: The lower bound of the price range (tick).
    :param upper_tick: The upper bound of the price range (tick).
    :param current_tick: The current price tick.
    :return: A tuple of (amount_base_asset, amount_quote_asset).
    """
    
    # get what is needed from the current state 
    current_tick = state_dict['pool']['current_tick']
    lower_tick = state_dict['pool']['desired_tick_range'][0]
    upper_tick = state_dict['pool']['desired_tick_range'][-1]

    # Convert ticks to prices
    lower_price = 1.0001 ** lower_tick
    upper_price = 1.0001 ** upper_tick
    current_price = 1.0001 ** current_tick

    # Assume a fixed amount of liquidity to provide
    # This is a simplified assumption; in practice, this would be based on how much liquidity you want to provide
    liquidity_amount = 1  # Example fixed liquidity amount

    if current_price <= lower_price:
        # Current price is at or below the lower bound of the range
        # Provide all in quote asset
        amount_base_asset = 0
        amount_quote_asset = liquidity_amount
    elif current_price >= upper_price:
        # Current price is at or above the upper bound of the range
        # Provide all in base asset
        amount_base_asset = liquidity_amount
        amount_quote_asset = 0
    else:
        # Current price is within the range
        # Calculate required amounts of base and quote assets
        # These calculations are simplified and assume a certain relationship between price and liquidity
        amount_base_asset = liquidity_amount / math.sqrt(current_price)
        amount_quote_asset = liquidity_amount * math.sqrt(current_price)

    return amount_base_asset, amount_quote_asset

# Example usage
amount_base, amount_quote = calculate_liquidity_amounts(state_dict)
print(f"Amount Base Asset: {amount_base}")
print(f"Amount Quote Asset: {amount_quote}")


Amount Base Asset: 21.60546636750988
Amount Quote Asset: 0.04628458293794536


#### figuring out how much needs to be transacted to wholly enter `current_tick` at the `desired_tick_range`

In [58]:
def figure_update_desired_bags_check_if_transaction_needed(state_dict): 
    """for now this takes a human_price
    # ###PAUL TODO: change `human_price` to `quote_price`
    """
    
    base_qty = state_dict['bags']['actual']['human_format']['base']
    quote_qty = state_dict['bags']['actual']['human_format']['quote']
    
    ratio_price = calculate_price_from_tick(state_dict['pool']['current_tick']) 
    base_in_quote = base_qty * ratio_price
    total_val_in_quote = base_in_quote + quote_qty

    desired_quote = total_val_in_quote/2
    desired_base = desired_quote/ratio_price
    
    state_dict['bags']['desired']['human_format']['base'] = desired_base
    state_dict['bags']['desired']['human_format']['quote'] = desired_quote 
    state_dict['bags']['desired']['wei_format']['base'] = convert_asset_human_amount_to_wei(desired_base, state_dict['bags']['decimal']['base'])
    state_dict['bags']['desired']['wei_format']['quote'] = convert_asset_human_amount_to_wei(desired_quote, state_dict['bags']['decimal']['quote'])
    
    return None 

In [59]:
figure_update_desired_bags_check_if_transaction_needed(state_dict)

In [60]:
state_dict['bags']['desired']

{'wei_format': {'base': 2.463172357092109e+19, 'quote': 5.276762061647821e+16},
 'human_format': {'base': 24.631723570921093, 'quote': 0.05276762061647821}}

##### figuring which router address to use

* manually executing a transaction
    * 0xec8b0f7ffe3ae75d7ffab09429e3675bb63503e4 ---- what appears in Rabby
    * 0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad ---- what arbiscan says this rabby addrss is identical to

      
* from uniswap documentation https://docs.uniswap.org/contracts/v3/reference/deployments
    * SwapRouter: 0xE592427A0AEce92De3Edee1F18E0157C05861564
    * SwapRouter02: 0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45
    * UniversalRouter: 0x3fC91A3afd70395Cd496C647d5a6CC9D4B2b7FAD
 


##### way to handle for slippage (suggested by chatGPT

In [ ]:
# Example: setting amount_out_min considering a 2% slippage

# Fetch the current price of GNS in terms of WETH from an oracle or a DEX interface
current_price_gns_per_weth = ...  # Replace with the actual current price

# Calculate the expected amount of GNS without slippage
expected_amount_out = amount_in * current_price_gns_per_weth

# Calculate the minimum amount you're willing to accept, considering slippage
slippage_tolerance = 0.02  # 2% slippage
amount_out_min = expected_amount_out * (1 - slippage_tolerance)

# Convert amount_out_min to wei if it's not already in wei
amount_out_min_wei = int(amount_out_min * 10**18)

In [ ]:
from web3 import Web3, Account
from eth_account.messages import encode_defunct

# Your wallet and private key
wallet_address = public_addrss

# Uniswap V3 Router Contract Address and ABI (use the appropriate Router ABI)
router_contract_address = '0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45'  # ###PAUL TODO: replace with addr via params 
router_abi = get_contract_abi(etherscan_api_key, router_contract_address)

# Create the Router contract instance
router_contract = w3.eth.contract(address=router_contract_address, abi=router_abi)

# Prepare the swap parameters (example for swapping WETH for GNS)
amount_in = 1000000000000000000  # Amount of WETH in wei you want to swap
amount_out_min = 1  # Minimum amount of GNS you want to receive (set considering slippage)
weth_address = weth_contract_address
gns_address = gns_contract_address
deadline = 1742680400  # Timestamp deadline for the transaction (example value)

# Swap function call (example, exact function depends on the Router's API)
swap_txn = router_contract.functions.swapExactTokensForTokens(
    amount_in,
    amount_out_min,
    [weth_address, gns_address],
    wallet_address,
    deadline
).buildTransaction({
    'from': wallet_address,
    'gas': 2000000,
    'gasPrice': w3.toWei('50', 'gwei'),
    'nonce': w3.eth.getTransactionCount(wallet_address),
})

# Sign the transaction
signed_txn = w3.eth.account.sign_transaction(swap_txn, private_key=private_key_str)

# Send the transaction
tx_hash = w3.eth.sendRawTransaction(signed_txn.rawTransaction)

print(f"Transaction hash: {tx_hash.hex()}")


In [ ]:
price

In [ ]:
# get_wallet_key(wallet=3)

# working liquidity looter class that gets near soruce 

In [ ]:
from web3 import Web3
import json

class LiquidityLooter:
    def __init__(self, rpc_url):
        self.web3 = Web3(Web3.HTTPProvider(rpc_url))
        if self.web3.is_connected():
            print("Connected to Ethereum blockchain.")
        else:
            raise Exception("Unable to connect to Ethereum blockchain.")

    def fetch_current_pool_price_range(self, pool_address, abi):
        # Convert the pool address to checksum format
        checksum_address = Web3.to_checksum_address(pool_address)

        # Create a contract object using the ABI and the pool's address
        pool_contract = self.web3.eth.contract(address=checksum_address, abi=abi)

        try:
            # Fetching slot0 which contains the current price and other data
            slot0 = pool_contract.functions.slot0().call()
            current_price = slot0[0]

            # Convert current price to a more readable format if necessary
            # This depends on how prices are represented in the contract

            # Return the current price (this can be expanded to include more data)
            return {"current_price": current_price}

        except Exception as e:
            print(f"An error occurred: {e}")
            return None

# Example usage
rpc_url = "https://arb1.arbitrum.io/rpc"  # Arbitrum RPC URL
loot_liquidity = LiquidityLooter(rpc_url)

# The contract address and ABI for the Uniswap v3 pool
pool_address = "0xc91b7b39bbb2c733f0e7459348fd0c80259c8471"
# abi = [...]  # Replace with the ABI provided

# Fetching the pool price range for the specific pool
price_range = loot_liquidity.fetch_current_pool_price_range(pool_address, pool_abi)
print(f"Price Range: {price_range}")


In [ ]:
loot_liquidity

In [ ]:
price_range

In [ ]:
sqrtPriceX96 = 3577156806314548018171497387
price_ratio = (sqrtPriceX96 / 2**96) ** 2

# If the tokens have different decimals, adjust the price ratio.
# For example, if token0 has 18 decimals and token1 has 6 decimals:
# price_ratio = price_ratio / (10**(18-6))

print("Price Ratio:", price_ratio)


* high level object needed 
    * `params`
    * `state_dict` 
* # work flow of this process
    * leave 0.01 ETH for gas. *I will just watch and refill manually for now* 
    * consider all funds that are note 0.01 ETH "for sale" for the liquidity pool
    * The strategy is as follows.
        * `lp_width` width in % of price for ceiling and floor price on LP
        * `p_rebalance1` proportion of `lp_width` needed to travel in either dimension to do an automatic new LP NFT
        * these live in config 
    * use uniswap price, calculate how much needed to trade on uniswap
    * 